# Analyzign Models Best Suited for Gaps Project

### Notebook Code Author: 
- Austin Youngren

### Data Collection Performed by: 
- Spencer Baloga Loufek 
- Ayush Adhikari

### Resources Used for Code Creation: 
- [Hugging Face](huggingface.co)
- [Facebook Research DPR GitHub Repository](https://github.com/facebookresearch/DPR)
- [Deepseek](https://www.deepseek.com/)

Deepseek AI chatbot was used due to limited access of DPR training documentation and examples.

## Imports

In [1]:
import gc
import re
import csv
import json
import torch
import wandb
import random
import pandas as pd
import torch.nn.functional as F

# from torch import nn
# from itertools import product
from torch.optim import AdamW
from torch.utils.data import DataLoader
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import precision_recall_fscore_support
from transformers import ( DPRQuestionEncoder, 
                            DPRQuestionEncoderTokenizer, 
                            DPRContextEncoder, 
                            DPRContextEncoderTokenizer,
                            get_linear_schedule_with_warmup )

pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None) 
pd.set_option('display.width', None) 
pd.set_option('display.max_colwidth', None) 
pd.set_option('display.max_seq_item', None)

C:\Users\ayoun\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(torch.__version__)  # e.g., 2.2.0+cu121
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.get_device_name(0))

2.5.1
True
NVIDIA GeForce RTX 3080


## Data Preprocessing

In [3]:
with open ('data/all-csu-codes.csv', 'r') as c_data:
    csv_reader = csv.reader(c_data) 
    courses_data = list(csv_reader)
    
courses_df = pd.DataFrame(columns=['Courses', 'Skills'])

for idx in range(0, len(courses_data)):
    skill_list = courses_data[idx][1:-1]
    skill_list = [skill.title() for skill in skill_list]
    skill_list = [re.sub(r'\b(vs|Vs)\b', 'VS', skill) for skill in skill_list]

    row = pd.DataFrame({'Courses': courses_data[idx][0], 'Skills':[skill_list]})
    courses_df = pd.concat([courses_df, row], ignore_index=True)

courses_df.to_csv('data/dpr_courses_data.csv', index=False)

In [4]:
with open('data/descriptions.txt', 'r') as j_data:
    csv_reader = csv.reader(j_data, delimiter='|')
    jobs_data = list(csv_reader)

jobs_df = pd.DataFrame(columns=['Job_Title', 'Job_Description', 'Required_Skills'])
                    
for row in jobs_data:
    if len(row) == 3:
        job_title = row[0].strip().strip('"') 

        job_description = row[1].strip().strip('"')
        job_description = re.sub(r'\bDESCRIPTION\b', '', job_description)
    
        skills = row[2].strip().strip('"')
        skill_list = [skill.strip().strip('"') for skill in skills.split(',')]
        cap_skill_list = [skill.title() for skill in skill_list]
        cleaned_skills = [re.sub(r'\s?\(.*?\)', '', skill) for skill in cap_skill_list]

        row = pd.DataFrame({'Job_Title': job_title, 'Job_Description': job_description, 'Required_Skills': [skill_list]})
        jobs_df = pd.concat([jobs_df, row], ignore_index=True)
            
jobs_df.to_csv('data/dpr_jobs.csv', index=False)

In [5]:
check_course = next(item for item in courses_data if item[0] == 'CS470')
print(check_course)
print('********************************************')
print(courses_df.head(1))

['CS470', 'Assembly', 'C Language', 'Digital Gates', 'Instruction Set Architectures', 'Number Representation']
********************************************
  Courses  \
0   CS462   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       Skills  
0  [3D Modeling, Animations, Assigning 3D Object Properties, Blender, Calculating Lights, Shades,

In [6]:
courses_df.head(1)

,Courses,Skills
0,CS462,"[3D Modeling, Animations, Assigning 3D Object Properties, Blender, Calculating Lights, Shades, And, Camera Rendering, Character Animation, Character Design, Collisions, Constructing 3D Scenes, Design 3D, Develop 3D, Event Triggering, Finite State Machine, Fourier Transform, Game Design, Geometric Image Manipulation, Human Aware Ai, Immersive 3D Worlds, Implementing Animation To Chara, Implementing Movement To Charac, Manipulate Lighting, Manipulate Rendering, Manipulating 3D Object Properti, Movement, Principles Of Lighting, Principles Of Rendering, Real Time Rendering Pipeline, Scene Composition, Scene Layout, Scripting Interactive Behaviors, Scripting Interactive Elements, Tangent Space, Unity, Vectors]"


In [7]:
check_job = [item for item in jobs_data if item[0] == 'Yahoo_Software_Dev_Engineer']
print(check_job)
print('********************************************')
print(jobs_df.head(1))

[['Yahoo_Software_Dev_Engineer', "Who We're Looking For- Junior Software Engineer We hire engineers who love the web, but can see its cracks and its future, too. We look for people who are exceptionally who are exceptionally imaginative, collaborative, and truly excited about tech. Our DSP Reporting team is currently looking for talented full-stack engineers to design, implement, and support robust, scalable, and high-quality reporting solutions Your Responsibilities - Develop and enhance a state-of-the-art reporting and analytics platform. - Build intuitive front-end UIs for reporting and analytics using React. - Develop microservices to power reporting and analytics solutions. - Write clean, maintainable, and performant code, including unit tests and refactoring when needed. - Collaborate with designers and developers to define and deliver new features. - Participate in system architecture reviews, code reviews, performance tuning, and production support. - Own and deliver projects r

In [8]:
jobs_df.head(1)

,Job_Title,Job_Description,Required_Skills
0,Adobe_AI_ML_Engineer,"The Opportunity?Adobe is seeking talented and passionate Software Engineer across all organizations to help plan, design, develop, and test software systems or applications for software enhancements and new products used in local, networked, cloud-based or Internet-related computer programs and products. What You'll Do - Develop high-performance, reliable, testable and maintainable code. - Participating in all aspects of software development activities, including design, coding, code review, testing, bug fixing, and code/API documentation. - Collaborate with engineers and participate in daily or weekly stand ups and meetings. - Grow with the support of your team and help others on the team grow by providing thoughtful feedback and uplifting those around you. - Work both independently and collaboratively within a fast-paced development team, with clear, positive, and constructive communication. - Additional responsibilities as needed based on specific role or team What You Need to Succeed - Bachelor's or Master's in Computer Science, Computer Engineering, Electrical Engineer, or equivalent experience required - 1-3+ years of experience in specific skill/field(s) - Proficient in programming languages such as Ruby, Python, Javascript, etc. - Strong technical background with analytical and problem-solving skills - Familiarity with client-side frameworks and libraries like React, Vue, Angular, Web Components. - Excellent problem solving and debugging skills, and direct experience with DevOps in a SaaS environment is a plus. - Customer focused and have real passion for quality and engineering excellence at scale. - Excellent communication and collaboration skills.","[Programming, Ruby, JavaScript, Python, Analytical skills, Knowledge sharing, Angular, Problem Solving Skills, React.js, Strong communication skills, Vue, Front-end frameworks, Collaboration, Communication, Development, Quality Assurance, Customer needs, Efficiency, Bug fixing, Code/API documentation, Code Review, Design, High performance, Reliability, Testing, Maintaining software applications, Feedback, Teamwork, Debugging, DevOps, Planning]"


In [9]:
del check_course, check_job, j_data, jobs_data, csv_reader, c_data, courses_data
gc.collect()
courses_df.shape, jobs_df.shape

((35, 2), (79, 3))

### Collect All Skills from CSU Course Data

In [10]:
def get_all_acquired_skills(courses_df):
    all_acquired_skills = set() 
    
    for skills in courses_df['Skills']:
        all_acquired_skills.update(skills)

    return all_acquired_skills

In [11]:
all_acquired_skills = get_all_acquired_skills(courses_df)
list(all_acquired_skills)[0:9]

['Webpack',
 'Bittorrent',
 'Grid Computing',
 'Non-Deterministic Polynomial Co',
 'Packet Switching',
 'Scripting Interactive Elements',
 'Combiner Functions',
 'Function Regression',
 'Mocking Concepts']

### Create Corpus for CSU Courses 

In [12]:
# def create_corpus(courses_df):
#     corpus = []
#     for idx, row in courses_df.iterrows():
#         document = {
#             "id": f"csu_course_{idx}",
#             "title": row['Courses'],
#             "text": f"Skills: {', '.join(row['Skills'])}",
#             "metadata": {"course_name": row['Courses'], "skills_acquired": row['Skills']}
#         }
#         corpus.append(document)
#     return corpus


# courses_corpus = create_corpus(courses_df)

# with open("data/dpr_course_corpus.json", "w") as f:
#     json.dump(courses_corpus, f, indent=4)

# json.dumps(courses_corpus[0], indent=4)

### Create the Training Dataset

In [13]:
def all_class_comparison(jobs_df, courses_df, all_acquired_skills):
    training_data = []
    course_entries = list(zip(courses_df["Courses"], courses_df["Skills"]))


    for _, job in jobs_df.iterrows():
        job_text = f"Job Title: {job['Job_Title']} -- Job Description: {job['Job_Description']}"
        job_skills = set(job["Required_Skills"])
        missing_skills = job_skills - all_acquired_skills
        
        pos_courses = []
        neg_courses = []

        for course, skills in course_entries:
            if any(skill in job_skills for skill in skills):
                pos_courses.append(f"[COURSE] {course} [SKILLS] {', '.join(skills)}")
            else:
                neg_courses.append(f"[COURSE] {course} [SKILLS] {', '.join(skills)}")

        pos_context = "\n".join(pos_courses) if pos_courses else "No relevant courses found."
        neg_context = "\n".join(neg_courses) if neg_courses else "No irrelevant courses found."

        if len(missing_skills) == 0:
            training_data.append({
                'query': job_text,
                'pos_context': pos_context,
                'neg_context':  neg_context,
                'answer': f"You qualify as an applicant for the job position, {job['Job_Title']}. No required skills are missing",
                'label': 0
            })
        else:
            training_data.append({
                'query': job_text,
                'pos_context': pos_context,
                'neg_context':  neg_context,
                'answer': f"Missing Skills: ".join(list(missing_skills)),
                'label': 1
            })
        
    return training_data

In [14]:
def compare_individual_course(jobs_df, courses_df, neg_per_pos=1):
    training_data = []
     
    for _, job in jobs_df.iterrows():
        job_text = f"Job Title: {job['Job_Title']} -- Job Description: {job['Job_Description']}"
        job_skills = set(job['Required_Skills'])  
        
        for _, course in courses_df.iterrows():
            course_entries = list(zip(course["Courses"], course["Skills"]))
            pos_courses = []
            neg_courses = []
            missing_skills = list(job_skills - set(course["Skills"]))
            
            for course, skills in course_entries:
                if any(skill in job_skills for skill in skills):
                    pos_courses.append(f"[COURSE] {course} [SKILLS] {', '.join(skills)}")
                else:
                    neg_courses.append(f"[COURSE] {course} [SKILLS] {', '.join(skills)}")

            pos_context = "\n".join(pos_courses) if pos_courses else "No relevant courses found."
            neg_context = "\n".join(neg_courses) if neg_courses else "No irrelevant courses found."

            if len(missing_skills) == 0:
                training_data.append({
                    'query': job_text,
                    'pos_context': pos_context,
                    'neg_context': neg_context,
                    'answer': f"You qualify as an applicant for the job position, {job['Job_Title']}. No required skills are missing",
                    'label': 0
                })
            else:
                training_data.append({
                    'query': job_text,
                    'pos_context': pos_context,
                    'neg_context': neg_context,
                    'answer': f"Missing Skills: ".join(list(missing_skills)),
                    'label': 1
                })

    return training_data

In [15]:
def create_schedule_data(jobs_df, schedules):
    training_data = []

    for _, job in jobs_df.iterrows():
        job_text = f"Job Title: {job['Job_Title']} -- Job Description: {job['Job_Description']}"
        job_skills = set(job["Required_Skills"])

        for sched in schedules:
            course_entries = list(zip(sched["Courses"], sched["Skills"]))
            pos_courses = []
            neg_courses = []
            sched_skills = set([item for sublist in sched["Skills"].tolist() for item in sublist])
            missing_skills = list(job_skills - sched_skills)

            for course, skills in course_entries:
                if any(skill in job_skills for skill in skills):
                    pos_courses.append(f"[COURSE] {course} [SKILLS] {', '.join(skills)}")
                else:
                    neg_courses.append(f"[COURSE] {course} [SKILLS] {', '.join(skills)}")

            pos_context = "\n".join(pos_courses) if pos_courses else "No relevant courses found."
            neg_context = "\n".join(neg_courses) if neg_courses else "No irrelevant courses found."

            if len(missing_skills) == 0:
                training_data.append({
                    'query': job_text,
                    'pos_context': pos_context,
                    'neg_context': neg_context,
                    'answer': f"You qualify as an applicant for the job position, {job['Job_Title']}. No required skills are missing",
                    'label': 0
                })
            else:
                training_data.append({
                    'query': job_text,
                    'pos_context': pos_context,
                    'neg_context': neg_context,
                    'answer': f"Missing Skills: ".join(list(missing_skills)),
                    'label': 1
                })

    return training_data


def get_courseloads(jobs_df, courses_df, number_of_schedules=20):
    core_classes = ['CS150', 'CS164', 'CS152', 'CS162', 'CS201', 'CS165', 'CS220', 
                    'CS270', 'CS250', 'CS314', 'CS370', 'CS320', 'CS214']
    
    elective_courses_df = courses_df[~courses_df['Courses'].isin(core_classes)]

    schedules_df = []
    used_schedules = set()

    while len(schedules_df) < number_of_schedules:
        l_4_courses = elective_courses_df[elective_courses_df['Courses'].str.startswith('CS4')]
        l_3_4_courses = elective_courses_df[elective_courses_df['Courses'].str.startswith('CS3') | elective_courses_df['Courses'].str.startswith('CS4')]
        other_courses = elective_courses_df[~elective_courses_df['Courses'].str.startswith('CS3') & ~elective_courses_df['Courses'].str.startswith('CS4')]

        l_4_sample = random.sample(l_4_courses['Courses'].tolist(), 2)

        l_3_4_filtered = l_3_4_courses[~l_3_4_courses['Courses'].isin(l_4_sample)]
        l_3_4_sample = random.sample(l_3_4_filtered['Courses'].tolist(), 2)

        all_sampled_courses = l_4_sample + l_3_4_sample
        other_courses_filtered = other_courses[~other_courses['Courses'].isin(all_sampled_courses)]
        other_sample = random.sample(other_courses_filtered['Courses'].tolist(), 1)

        sched_courses = core_classes + l_4_sample + l_3_4_sample + other_sample
        sched_df = courses_df[courses_df['Courses'].isin(sched_courses)].copy()

        sched_tuple = tuple(sorted(sched_df['Courses'].tolist()))
        if sched_tuple not in used_schedules:
            schedules_df.append(sched_df)
            used_schedules.add(sched_tuple)

    training_data = create_schedule_data(jobs_df, schedules_df)

    return training_data

In [16]:
def create_training_data(jobs_df, courses_df, all_acquired_skills):
    training_data = all_class_comparison(jobs_df, courses_df, all_acquired_skills)
    print(f"All: {training_data[-1]}\n")

    training_data = training_data + compare_individual_course(jobs_df, courses_df)
    print(f"Individual: {training_data[-1]}\n")
    
    training_data = training_data + get_courseloads(jobs_df, courses_df)
    print(f"Course load: {training_data[-1]}\n")

    return training_data

training_data = create_training_data(jobs_df, courses_df, all_acquired_skills)

All: {'query': "Job Title: Yahoo_Software_Dev_Engineer -- Job Description: Who We're Looking For- Junior Software Engineer We hire engineers who love the web, but can see its cracks and its future, too. We look for people who are exceptionally who are exceptionally imaginative, collaborative, and truly excited about tech. Our DSP Reporting team is currently looking for talented full-stack engineers to design, implement, and support robust, scalable, and high-quality reporting solutions Your Responsibilities - Develop and enhance a state-of-the-art reporting and analytics platform. - Build intuitive front-end UIs for reporting and analytics using React. - Develop microservices to power reporting and analytics solutions. - Write clean, maintainable, and performant code, including unit tests and refactoring when needed. - Collaborate with designers and developers to define and deliver new features. - Participate in system architecture reviews, code reviews, performance tuning, and product

In [17]:
td_df = pd.DataFrame(training_data, columns=['question', 'pos_context', 'neg_context', 'answer', 'label'])
print(td_df.shape)

td_df.to_csv('data/dpr_training_data.csv', index=False)

del training_data, jobs_df, courses_df
gc.collect()

(4424, 5)


23

In [18]:
td_df.head(1)

question  \
0       NaN   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

## Question (Jobs) and Context (Courses) Encoding for DPR Training

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# question_encoder = DPRQuestionEncoder.from_pretrained('facebook/dpr-question_encoder-single-nq-base')
# context_encoder = DPRContextEncoder.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')

# question_encoder = question_encoder.to(device)
# context_encoder = context_encoder.to(device)

# pos_weight = torch.tensor([(len(td_df) - td_df['label'].sum()) / td_df['label'].sum()]).to(device)
# loss_fn = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)
# print(td_df['label'].value_counts())

In [20]:
question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained('facebook/dpr-question_encoder-single-nq-base')
context_tokenizer = DPRContextEncoderTokenizer.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')

def encode_data(batch):
    max_length = 512

    question_texts = batch['question'] if isinstance(batch['question'], list) else [str(batch['question'])]
    question_texts = [str(text) for text in question_texts]
    
    pos_contexts = batch['pos_context'] if isinstance(batch['pos_context'], list) else [str(batch['pos_context'])]
    neg_contexts = batch['neg_context'] if isinstance(batch['neg_context'], list) else [str(batch['neg_context'])]
    
    pos_contexts = [str(text) for text in pos_contexts]
    neg_contexts = [str(text) for text in neg_contexts]

    question_encodings = question_tokenizer(
        question_texts, 
        padding="max_length", 
        truncation=True, 
        max_length=max_length, 
        return_tensors="pt"
    )
    pos_context_encodings = context_tokenizer(
        pos_contexts, 
        padding="max_length", 
        truncation=True, 
        max_length=max_length, 
        return_tensors="pt"
    )
    neg_context_encodings = context_tokenizer(
        neg_contexts, 
        padding="max_length", 
        truncation=True, 
        max_length=max_length, 
        return_tensors="pt"
    )

    # Handle labels (unchanged)
    if 'label' in batch:
        labels = torch.tensor(batch['label']).to(torch.float16)
    else:
        labels = torch.zeros(len(question_texts), dtype=torch.long)
        
    return {
        'question_input_ids': question_encodings['input_ids'],
        'question_attention_mask': question_encodings['attention_mask'],
        'pos_context_input_ids': pos_context_encodings['input_ids'],
        'pos_context_attention_mask': pos_context_encodings['attention_mask'],
        'neg_context_input_ids': neg_context_encodings['input_ids'],
        'neg_context_attention_mask': neg_context_encodings['attention_mask'],
        'labels': labels
    }

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.


In [21]:
train_df, temp_df = train_test_split(td_df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "validation": Dataset.from_pandas(val_df),
    "test": Dataset.from_pandas(test_df)
})

encoded_dataset = dataset.map(
    encode_data, 
    batched=True,
    remove_columns=dataset["train"].column_names  # Remove original columns
)

encoded_dataset.set_format(type='torch', columns=[
    'question_input_ids',
    'question_attention_mask',
    'pos_context_input_ids',
    'pos_context_attention_mask',
    'neg_context_input_ids',
    'neg_context_attention_mask',
    'labels'
])


Map: 100%|██████████| 664/664 [00:05<00:00, 112.94 examples/s]


## WandB Evaluation Setup

In [22]:
key_file = rf'D:\Development\cs580\CSU-Industry-Skills\dpr-model\WANDB_API_KEY.txt' 

with open(key_file, "r") as f:
    api_key = f.read().strip()

wandb.login(key=api_key)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\ayoun\_netrc
wandb: Currently logged in as: ayoungren94 (ayoungren-colostate) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [23]:
def compute_metrics(preds, labels):
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels.cpu(), preds.cpu(), average='binary', zero_division=0
    )
    acc = (preds == labels).float().mean()
    return {
        'accuracy': acc.item(),
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [24]:
def evaluate(model_tuple, dataloader, device):
    question_encoder, context_encoder = model_tuple
    question_encoder.eval()
    context_encoder.eval()
    
    all_preds, all_labels = [], []
    
    for batch in dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        
        query_out = question_encoder(
            input_ids=batch['question_input_ids'],
            attention_mask=batch['question_attention_mask']
        )
        pos_ctx_out = context_encoder(
            input_ids=batch['pos_context_input_ids'],
            attention_mask=batch['pos_context_attention_mask']
        )
        neg_ctx_out = context_encoder(
            input_ids=batch['neg_context_input_ids'],
            attention_mask=batch['neg_context_attention_mask']
        )

        pos_scores = torch.sum(query_out.pooler_output * pos_ctx_out.pooler_output, dim=1)
        neg_scores = torch.sum(query_out.pooler_output * neg_ctx_out.pooler_output, dim=1)
        
        preds = torch.stack([pos_scores, neg_scores], dim=1).argmax(dim=1)

        if preds.dim() == 0:
            preds = preds.unsqueeze(0)
        if batch['labels'].dim() == 0:
            batch['labels'] = batch['labels'].unsqueeze(0)
        
        all_preds.append(preds.cpu())
        all_labels.append(batch['labels'].cpu())

    if not all_preds or not all_labels:
        raise ValueError("No predictions or labels collected during evaluation.")
    
    return compute_metrics(torch.cat(all_preds), torch.cat(all_labels))

In [25]:
torch.cuda.empty_cache()
gc.collect()

159

In [26]:
def contrastive_loss(pos_scores, neg_scores, labels, margin=0.2):
    loss = torch.where(
        labels == 0,
        torch.relu(margin - pos_scores + neg_scores),  
        torch.relu(pos_scores - neg_scores + margin)
    )
    return loss.mean()

In [27]:
def train_dpr(config):
    model_name = f"dpr--lr:{config['lr']}--{config['batch_size']}--wd:{config['weight_decay']}"
    run = wandb.init(
        project="Gaps-DPR",
        entity="ayoungren-colostate",
        name= model_name,
        config={
            'learning_rate': config['lr'],
            'batch_size': config['batch_size'],
            'weight_decay': config['weight_decay'],
        }
    )

    torch.cuda.empty_cache()
    gc.collect()

    scaler = torch.amp.GradScaler('cuda',)
    autocast = torch.amp.autocast(device_type='cuda', dtype=torch.float16, enabled=True)
    
    question_encoder = DPRQuestionEncoder.from_pretrained('facebook/dpr-question_encoder-single-nq-base').to(device)
    context_encoder = DPRContextEncoder.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base').to(device)
    
    grad_accum_steps = max(1, config['batch_size']//8)
    effective_batch = config['batch_size'] // grad_accum_steps

    train_loader = DataLoader(
        encoded_dataset["train"], 
        batch_size=effective_batch, 
        shuffle=True,
        pin_memory=True,
        num_workers=2,        
        persistent_workers=True
    )
    
    val_loader = DataLoader(
        encoded_dataset["validation"], 
        batch_size=effective_batch, 
        shuffle=False,
        pin_memory=True
    )

    optimizer = AdamW(
        list(question_encoder.parameters()) + list(context_encoder.parameters()),
        lr=config['lr'],
        weight_decay=config['weight_decay']
    )

    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=config['warmup_steps'],
        num_training_steps=len(train_loader) * config['epochs'] // grad_accum_steps
    )

    best_val_f1 = 0
    early_stop_counter = 0
    patience = 30

    for epoch in range(1, config['epochs']+1):
        print(f"\nBeginning Epoch {epoch} for model architecture {model_name}")
        question_encoder.train()
        context_encoder.train()
        
        epoch_loss = 0
        all_preds, all_labels = [], []
        optimizer.zero_grad()

        for step, batch in enumerate(train_loader):
            # print(torch.cuda.memory_summary())
            batch = {k: v.to(device, non_blocking=True) for k, v in batch.items()}
            
            with autocast:
                query_out = question_encoder(
                    input_ids=batch['question_input_ids'],
                    attention_mask=batch['question_attention_mask']
                )
                pos_ctx_out = context_encoder(
                    input_ids=batch['pos_context_input_ids'],
                    attention_mask=batch['pos_context_attention_mask']
                )
                neg_ctx_out = context_encoder(
                    input_ids=batch['neg_context_input_ids'],
                    attention_mask=batch['neg_context_attention_mask']
                )

                pos_sim = torch.sum(query_out.pooler_output * pos_ctx_out.pooler_output, dim=1)
                neg_sim = torch.sum(query_out.pooler_output * neg_ctx_out.pooler_output, dim=1)

                targets = batch['labels'].float()
                loss = contrastive_loss(pos_sim, neg_sim, targets) / grad_accum_steps

                scores = torch.stack([pos_sim, neg_sim], dim=1)
                preds = scores.argmax(dim=1)

            scaler.scale(loss).backward()

            if (step + 1) % grad_accum_steps == 0:
                scaler.step(optimizer)
                scaler.update() 
                scheduler.step() 
                optimizer.zero_grad()

            epoch_loss += loss.item() * grad_accum_steps
            
            with torch.no_grad():
                preds = scores.argmax(dim=-1).detach().cpu()
                all_preds.append(preds)
                all_labels.append(batch['labels'].cpu())

            if step % 10 == 0:
                run.log({
                    "batch_loss": loss.item() * grad_accum_steps,
                    "batch_lr": scheduler.get_last_lr()[0],
                    "gpu_mem": torch.cuda.memory_allocated()/1e9
                }, commit=False) 
                torch.cuda.empty_cache()
                gc.collect()

        train_metrics = compute_metrics(torch.cat(all_preds), torch.cat(all_labels))
        avg_train_loss = epoch_loss / len(train_loader)

        val_metrics = evaluate((question_encoder, context_encoder), val_loader, device)

        run.log({
            "epoch": epoch,
            "train_loss": avg_train_loss,
            "train_accuracy": train_metrics['accuracy'],
            "train_f1": train_metrics['f1'],
            "train_precision": train_metrics['precision'],
            "train_recall": train_metrics['recall'],
            "val_accuracy": val_metrics['accuracy'],
            "val_f1": val_metrics['f1'],
            "val_precision": val_metrics['precision'],
            "val_recall": val_metrics['recall'],
            "learning_rate": scheduler.get_last_lr()[0],
            "gpu_mem": torch.cuda.memory_allocated()/1e9
        }, commit=True)

        if val_metrics['f1'] > best_val_f1:
            best_val_f1 = val_metrics['f1']
            early_stop_counter = 0
            torch.save({
                "question_encoder": question_encoder.state_dict(),
                "context_encoder": context_encoder.state_dict(),
                "config": config,
                "val_metrics": val_metrics
            }, f"best_dpr_model_{wandb.run.id}.pt")
        else:
            early_stop_counter += 1
            if early_stop_counter >= patience:
                print(f"Early stopping at epoch {epoch}")
                break

    checkpoint = torch.load(f"best_dpr_model_{wandb.run.id}.pt", map_location=device)
    question_encoder.load_state_dict(checkpoint["question_encoder"])
    context_encoder.load_state_dict(checkpoint["context_encoder"])
    
    test_metrics = evaluate((question_encoder, context_encoder), val_loader, device)
    run.log({
        "val_accuracy": test_metrics['accuracy'],
        "val_recall": test_metrics['recall'],
        "val_precision": test_metrics['precision'],
        "val_f1": test_metrics['f1'],
        "final_gpu_mem": torch.cuda.memory_allocated()/1e9
    })
    run.finish()

    return test_metrics['f1']

In [ ]:
hyperparams = {
    'epochs': [150],
    'lr': [2e-5, 3e-5, 4e-4, 4e-5],
    'batch_size': [128, 64, 32, 16],
    'weight_decay': [0.01],
    'warmup_steps': [500]
}

best_score = 0
best_config = None

for config in ParameterGrid(hyperparams):
    print(f"Training with config: {config}")
    current_score = train_dpr(config)
    
    if current_score > best_score:
        best_score = current_score
        best_config = config
        print(f"New best config: {best_config} with F1: {best_score}")

print(f"Best configuration: {best_config}")
print(f"Best validation F1: {best_score}")

Training with config: {'batch_size': 128, 'epochs': 150, 'lr': 2e-05, 'warmup_steps': 500, 'weight_decay': 0.01}


Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the


Beginning Epoch 1 for model architecture dpr--lr:2e-05--128--wd:0.01


c:\Users\ayoun\Miniconda3\envs\gaps\Lib\site-packages\torch\optim\lr_scheduler.py:224: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


In [ ]:
def evaluate_final_model(run_id):
    """Run this separately after selecting best run"""
    run = wandb.init(project="Gaps-DPR", name=f"final-test-{run_id}")
    
    checkpoint = torch.load(f"best_dpr_model_{run_id}.pt")
    

    question_encoder = DPRQuestionEncoder.from_pretrained('facebook/dpr-question_encoder-single-nq-base').to(device)
    context_encoder = DPRContextEncoder.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base').to(device)
    question_encoder.load_state_dict(checkpoint["question_encoder"])
    context_encoder.load_state_dict(checkpoint["context_encoder"])
    
    test_loader = DataLoader(
        encoded_dataset["test"], 
        batch_size=checkpoint["config"]["batch_size"], 
        shuffle=False,
        pin_memory=True,
        num_workers=4,
        persistent_workers=True
    )

    with torch.inference_mode():  
        test_metrics = evaluate((question_encoder, context_encoder), test_loader, device)
    
    run.log({
        "final_test_accuracy": test_metrics['accuracy'],
        "final_test_f1": test_metrics['f1']
    })
    run.finish()
    return test_metrics

In [ ]:
if best_config:
    print("\n=== Running Final Test Evaluation ===")
    test_results = evaluate_final_model(config)
    print(f"Final Test F1: {test_results['f1']:.4f}")